In [1]:
from data_loader import load_dataset1, load_dataset2
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# load and pre-process data
# mild imbalance in classes but should be fine
x1, y1 = load_dataset1()
x2, y2 = load_dataset2()
scalar = StandardScaler(copy=False)
scalar.fit(x1)
scalar.transform(x1)
scalar.fit(x2)

# binary var
scalar.mean_[4] = 0
scalar.var_[4] = 1
scalar.scale_[4] = 1
scalar.transform(x2)

# generate train and test split - NOTE: we do not use our test split at all until we have decided that we have our best model (which we will decide based on the validation
# loss)
train_x1, test_x1, train_y1, test_y1 = train_test_split(
    x1, y1, test_size=0.2, random_state=42
)
train_x2, test_x2, train_y2, test_y2 = train_test_split(
    x2, y2, test_size=0.2, random_state=42
)

In [3]:
# define params, model, and grid search
params = {
    "C": [0.1, 1, 10, 100],
    "kernel": ("linear", "poly", "rbf"),
    "degree": np.linspace(1, 8, 2, dtype="int"),
    "gamma": [0.0001, 0.001, 0.1, 1],
}
clf_1 = SVC(random_state=42, verbose=0)
clf_2 = SVC(random_state=42, verbose=0)
grid_search_1 = GridSearchCV(
    clf_1,
    params,
    cv=10,
    scoring=("accuracy", "f1", "precision", "recall", "roc_auc"),
    refit=False,
    verbose=2,
    n_jobs=-1,
    error_score=-1,
    return_train_score = True
)
grid_search_2 = GridSearchCV(
    clf_2,
    params,
    cv=10,
    scoring=("accuracy", "f1", "precision", "recall", "roc_auc"),
    refit=False,
    verbose=2,
    n_jobs=-1,
    error_score=-1,
    return_train_score = True
)

In [4]:
# run grid search, write results to csv file
grid_search_1.fit(train_x1, train_y1)
df = pd.DataFrame(grid_search_1.cv_results_)
df.to_csv("./results/svm_classifier_grid_search_data_set_1.csv")

grid_search_2.fit(train_x2, train_y2)
df = pd.DataFrame(grid_search_2.cv_results_)
df.to_csv("./results/svm_classifier_grid_search_data_set_2.csv")

Fitting 10 folds for each of 96 candidates, totalling 960 fits


/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

Fitting 10 folds for each of 96 candidates, totalling 960 fits


/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

[CV] END .......C=0.1, degree=1, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .........C=0.1, degree=1, gamma=0.0001, kernel=poly; total time=   0.0s
[CV] END ..........C=0.1, degree=1, gamma=0.0001, kernel=rbf; total time=   0.0s
[CV] END ........C=0.1, degree=1, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END ..........C=0.1, degree=1, gamma=0.001, kernel=poly; total time=   0.0s
[CV] END ...........C=0.1, degree=1, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END .............C=0.1, degree=1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .............C=0.1, degree=1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ........C=0.1, degree=8, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, degree=8, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, degree=8, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, degree=8, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END .............C=0.1,

/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

[CV] END .......C=0.1, degree=1, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .........C=0.1, degree=1, gamma=0.0001, kernel=poly; total time=   0.0s
[CV] END ..........C=0.1, degree=1, gamma=0.0001, kernel=rbf; total time=   0.0s
[CV] END ........C=0.1, degree=1, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END ..........C=0.1, degree=1, gamma=0.001, kernel=poly; total time=   0.0s
[CV] END ...........C=0.1, degree=1, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ............C=0.1, degree=1, gamma=0.1, kernel=poly; total time=   0.0s
[CV] END ..............C=0.1, degree=1, gamma=1, kernel=poly; total time=   0.0s
[CV] END ..............C=0.1, degree=1, gamma=1, kernel=poly; total time=   0.0s
[CV] END .......C=0.1, degree=8, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .......C=0.1, degree=8, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, degree=8, gamma=0.001, kernel=linear; total time=   0.0s
[CV] END ........C=0.1, degr

/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joyz/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning